## Import required libraries:

In [1]:
import pandas as pd

## Load the excel data:

In [2]:
ta = pd.read_excel('data/meta/Transit_Agencies_for_Visualization.xlsx', sheet_name='TC AgencyList')
upt = pd.read_excel('data/ntd/TS2.2TimeSeriesSysWideOpexpSvc_2.xlsx', sheet_name='UPT')

## Clean the data

In [3]:
# Remove missing NTD ID's
ta = ta.dropna(how='all')
upt = upt.dropna(subset=['NTD ID'])

# Combine project ID's
ta['Project ID'] = ta['Project ID'].combine_first(ta['"Other" primary Project ID'])

## Merge the data

In [9]:
# Merging dataframes
merge = pd.merge(ta, upt, how='left', on='NTD ID')
group = merge.groupby('Project ID')
group.sum().to_csv('test.csv')